<a href="https://colab.research.google.com/github/institutohumai/cursos-python/blob/master/BasesDeDatos/1_Intro_DBs/intro_sql.ipynb"> <img src='https://colab.research.google.com/assets/colab-badge.svg'/> </a>

# SQL I

In [1]:
# install
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql

import sqlalchemy
import pandas as pd

engine = sqlalchemy.create_engine('postgresql+psycopg2://@/postgres')

def exec_sql(query):
  text_query = sqlalchemy.sql.text(query)

  with engine.connect() as conn:
    res = conn.execute(text_query)
    if query.strip().lower().startswith('select'):
      return pd.DataFrame(res)
    else:
      conn.commit()

 * Starting PostgreSQL 12 database server
   ...done.
CREATE ROLE


### Crear base de datos

In [2]:
create_schema = 'CREATE SCHEMA IF NOT EXISTS humai'

exec_sql(create_schema)

## DDL - Crear tablas

### Tabla Customers



```
CREATE TABLE IF NOT EXISTS humai.Customers (
  customer_id INT NOT NULL,
  customer_name VARCHAR(50) NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE,
  PRIMARY KEY (customer_id)
  );
```



In [3]:
create_customers = """
CREATE TABLE IF NOT EXISTS humai.Customers (
  customer_id INT NOT NULL,
  customer_name VARCHAR(50) NOT NULL,
  fecha_inicio DATE NOT NULL,
  fecha_fin DATE,
  PRIMARY KEY (customer_id));
"""

exec_sql(create_customers)

### Tabla Orders



```
CREATE TABLE IF NOT EXISTS humai.Orders (
  order_id INT NOT NULL,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  order_price DECIMAL(8,2),
  PRIMARY KEY (order_id),
FOREIGN KEY (customer_id) REFERENCES humai.Customers(customer_id)
  );
```



In [4]:
create_orders = """
CREATE TABLE IF NOT EXISTS humai.Orders (
  order_id INT NOT NULL,
  customer_id INT NOT NULL,
  order_date DATE NOT NULL,
  order_price DECIMAL(8,2),
  PRIMARY KEY (order_id),
  FOREIGN KEY (customer_id) REFERENCES humai.Customers(customer_id)
);
"""

exec_sql(create_orders)

### Tabla Shipments

```
CREATE TABLE IF NOT EXISTS humai.Shipments (
  shipment_id INT NOT NULL,
  order_id INT NOT NULL,
  shipment_date DATE NOT NULL,
  shipment_city VARCHAR(50),
  PRIMARY KEY (shipment_id),
  FOREIGN KEY (order_id) REFERENCES humai.Orders(order_id)
  );

```



In [5]:
create_shipments = """
CREATE TABLE IF NOT EXISTS humai.Shipments (
  shipment_id INT NOT NULL,
  order_id INT NOT NULL,
  shipment_date DATE NOT NULL,
  shipment_city VARCHAR(50),
  PRIMARY KEY (shipment_id),
  FOREIGN KEY (order_id) REFERENCES humai.Orders(order_id)
);
"""

exec_sql(create_shipments)

### Check que se hayan creado las tablas

In [6]:
exec_sql("SELECT * FROM information_schema.tables WHERE table_schema = 'humai';")

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action
0,postgres,humai,customers,BASE TABLE,None,None,None,None,None,YES,NO,None
1,postgres,humai,orders,BASE TABLE,None,None,None,None,None,YES,NO,None
2,postgres,humai,shipments,BASE TABLE,None,None,None,None,None,YES,NO,None


## DML - Insertar datos ficticios



```
INSERT INTO humai.Customers
VALUES
(1, 'Eugenio', '08/21/1998', Null),
(2, 'Mario', '05/05/2005', Null),
(3, 'Pedro', '03/08/2020', '02/05/2022')
;
```



In [7]:
insert_customers = """
INSERT INTO humai.Customers VALUES
  (1, 'Eugenio', '08/21/1998', Null),
  (2, 'Mario', '05/05/2005', Null),
  (3, 'Pedro', '03/08/2020', '02/05/2022')
"""

exec_sql(insert_customers)



```
INSERT INTO humai.Orders
VALUES
(1, 1, '06/05/2022', 45),
(2, 1, '06/05/2021', 60),
(3, 1, '06/06/2022', 70),
(4, 2, '01/05/2022', 5),
(5, 3, '06/10/2022', 145),
(6, 3, '03/02/2022', 2)
;
```



In [8]:
insert_orders = """
INSERT INTO humai.Orders VALUES
  (1, 1, '06/05/2022', 45),
  (2, 1, '06/05/2021', 60),
  (3, 1, '06/06/2022', 70),
  (4, 2, '01/05/2022', 5),
  (5, 3, '06/10/2022', 145),
  (6, 3, '03/02/2022', 2);
"""

exec_sql(insert_orders)


```
INSERT INTO humai.Shipments
VALUES
(1, 1, '06/06/2022', 'Belgrano'),
(2, 2, '06/06/2021', 'Mar del Plata'),
(3, 3, '06/10/2022', 'Belgrano'),
(4, 4, '02/05/2022', 'San Isidro'),
(5, 5, '06/15/2022', 'Belgrano'),
(6, 6, '03/05/2022', 'Mar del Plata')
;
```





In [9]:
insert_shipments = """
INSERT INTO humai.Shipments VALUES
  (1, 1, '06/06/2022', 'Belgrano'),
  (2, 2, '06/06/2021', 'Mar del Plata'),
  (3, 3, '06/10/2022', 'Belgrano'),
  (4, 4, '02/05/2022', 'San Isidro'),
  (5, 5, '06/15/2022', 'Belgrano'),
  (6, 6, '03/05/2022', 'Mar del Plata');
"""

exec_sql(insert_shipments)

## DDL - Consulas SELECT

1. Retorna todos los registros de la tabla Customers.



```
SELECT *
FROM humai.Customers;
```



In [10]:
select_customers = 'SELECT * FROM humai.Customers;'

exec_sql(select_customers)

,customer_id,customer_name,fecha_inicio,fecha_fin
0,1,Eugenio,1998-08-21,None
1,2,Mario,2005-05-05,None
2,3,Pedro,2020-03-08,2022-02-05


2. Retorna la cantidad de envios por shipment_city



```
SELECT shipment_city as "Barrio", count(*) as "Envios"
FROM humai.Shipments
GROUP BY shipment_city
```



In [11]:
select_shipment = 'SELECT shipment_city as "Barrio", count(*) as "Envios" FROM humai.Shipments  GROUP BY shipment_city'

exec_sql(select_shipment)

,Barrio,Envios
0,Belgrano,3
1,Mar del Plata,2
2,San Isidro,1


3. Retorna la cantidad de envios a Belgrano



```
SELECT count(*) as "Envios a Belgrano"
FROM humai.Shipments  
WHERE shipment_city = 'Belgrano'
```



In [12]:
exec_sql("SELECT count(*) as \"Envios a Belgrano\" FROM humai.Shipments  WHERE shipment_city = 'Belgrano';")

,Envios a Belgrano
0,3


4. Retorna todas las ordenes mayores a $50



```
SELECT *
FROM humai.Orders
WHERE order_price > 50;
```



In [13]:
exec_sql("SELECT * FROM humai.Orders WHERE order_price > 50;")

,order_id,customer_id,order_date,order_price
0,2,1,2021-06-05,60.00
1,3,1,2022-06-06,70.00
2,5,3,2022-06-10,145.00


5. Retorna el cliente que más dinero gasto



```
SELECT customer_id, sum(order_price) as "Total gastado"
FROM humai.Orders
GROUP BY customer_id
ORDER BY sum(order_price) DESC
LIMIT 1
;
```



In [14]:
exec_sql("SELECT customer_id, sum(order_price) as \"Total gastado\" FROM humai.Orders GROUP BY customer_id ORDER BY sum(order_price) DESC LIMIT 1;")

,customer_id,Total gastado
0,1,175.00


6. Y que si queremos ver el nombre del cliente? Dato que se encuentra en otra tabla.



```
SELECT c.customer_name, sum(order_price) as "Total gastado"
FROM humai.Orders o
FULL JOIN humai.Customers c
ON o.customer_id = c.customer_id
GROUP BY c.customer_name
ORDER BY sum(order_price) DESC
LIMIT 1;
```



In [15]:
select = """
SELECT c.customer_name, sum(order_price) as \"Total gastado\"
  FROM humai.Orders o
  FULL JOIN humai.Customers c
  ON o.customer_id = c.customer_id
  GROUP BY c.customer_name
  ORDER BY sum(order_price) DESC
  LIMIT 1;
"""

exec_sql(select)

,customer_name,Total gastado
0,Eugenio,175.00


7. Retornar la duración promedio de los clientes que se dieron de baja.



```
SELECT CAST(AVG(fecha_fin-fecha_inicio) AS INT) AS "Promedio Total dias"
FROM humai.Customers
WHERE fecha_fin IS NOT NULL
```



In [16]:
exec_sql("SELECT CAST(AVG(fecha_fin-fecha_inicio) AS INT) AS \"Promedio Total dias\" FROM humai.Customers WHERE fecha_fin IS NOT NULL")

,Promedio Total dias
0,699


8. Retornar los clientes que tienen una 'e' en el nombre.


```
SELECT customer_name
FROM humai.Customers
WHERE customer_name LIKE '%e%';

```



In [17]:
exec_sql("SELECT customer_name FROM humai.Customers WHERE customer_name LIKE '%e%';")

,customer_name
0,Eugenio
1,Pedro
